In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image   


notebook_dir = os.path.abspath("")

images_dir = os.path.join(os.path.abspath(os.path.join(notebook_dir,"data", "test_images")))

print(images_dir)

list_images = os.listdir(images_dir)
i = 0
for image_name in list_images:
    print(f"image {i}: {image_name}\n")
    i += 1


## 2D FFT Analysis on Basic Matrices

In [2]:

def plot_fft_results(amplitude, phase):
    """Plot original data, FFT amplitude, and FFT phase"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    

    im1 = ax1.imshow(np.log10(amplitude + 1), cmap='gray')
    ax1.set_title('FFT Amplitude (log scale)')
    
    im2 = ax2.imshow(phase, cmap='gray', vmin=-np.pi, vmax=np.pi)
    ax2.set_title('FFT Phase')
    
    plt.tight_layout()


def plot_images(np_images, titles=None, fig_size=(10, 10)):
    if isinstance(np_images, np.ndarray) and len(np_images.shape) in [2, 3]:
        np_images = [np_images]
    
    if titles is not None and not isinstance(titles, (list, tuple)):
        titles = [titles]
    
    n_images = len(np_images)
    fig, axes = plt.subplots(1, n_images, figsize=fig_size)
    
    if n_images == 1:
        axes = [axes]
    
    for i, (img, ax) in enumerate(zip(np_images, axes)):
        if len(img.shape) == 3 and img.shape[2] == 1:
            img = img[:,:,0]
        ax.imshow(img, cmap='gray' if len(img.shape) == 2 else None)
        ax.axis('off')
        if titles is not None and i < len(titles):
            ax.set_title(titles[i])
    
    plt.show()

    

In [ ]:
sample_array = np.array([
    [10, 10, 10],
    [20, 20, 20],
    [30, 30, 30],
])

fft_array = np.fft.fft2(sample_array)

print(fft_array)

magnitudes = np.abs(fft_array)

n_log_magnitudes = np.log10(magnitudes + 1)


angle = np.angle(fft_array)

inverse_fft = np.fft.ifft2(fft_array).real

plot_images([sample_array, n_log_magnitudes, angle, inverse_fft], ['Original Image', 'FFT Magnitude (log 10 scale)', 'FFT Phase', 'Inverse FFT'])



In [ ]:
np_shifting = np.fft.fftshift(fft_array)   


nlog_shifting = np.log10(np.abs(np_shifting) + 1)
shifted_magnitudes = np.abs(nlog_shifting)
shifted_angle = np.angle(fft_array)

plot_images([sample_array, shifted_magnitudes, shifted_angle], ['Original Image', 'FFT Magnitude (log scale)', 'FFT Phase'])



In [ ]:

inverse_fft = np.abs(np.fft.ifft2(fft_array)).astype(int)

print(inverse_fft)

## 2D FFT Analysis on Basic Image

In [6]:
import numpy as np
import cv2

def find_center(points):
    all_points = np.array([point for line in points for point in line])
    center = np.mean(all_points, axis=0)
    return center

def rotate_geometric_shape(points, angle):
    angle_rad = np.deg2rad(angle)
    
    center = find_center(points)
    
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])
    
    rotated_points = []
    for p1, p2 in points:
        p1 = np.array(p1)
        p2 = np.array(p2)
        
        p1_centered = p1 - center
        p2_centered = p2 - center
        
        p1_rotated = np.dot(rotation_matrix, p1_centered)
        p2_rotated = np.dot(rotation_matrix, p2_centered)
        
        p1_final = p1_rotated + center
        p2_final = p2_rotated + center
        
        p1_final = p1_final.astype(int)
        p2_final = p2_final.astype(int)
        
        rotated_points.append([p1_final, p2_final])
    
    return rotated_points

def shift_geometric_shape(points, shift_x, shift_y):
    shifted_points = []
    shift_vector = np.array([shift_x, shift_y])
    
    for p1, p2 in points:
        p1 = np.array(p1)
        p2 = np.array(p2)
        
        p1_shifted = p1 + shift_vector
        p2_shifted = p2 + shift_vector
        
        p1_shifted = p1_shifted.astype(int)
        p2_shifted = p2_shifted.astype(int)
        
        shifted_points.append([p1_shifted, p2_shifted])
    
    return shifted_points

def draw_image(points, 
               width: int = 500, 
               height: int = 500) -> np.ndarray:
    img = np.zeros((height, width), dtype=np.uint8)
    
    vertices = []
    edges = set()
    
    for p1, p2 in points:
        p1 = tuple(map(int, p1))
        p2 = tuple(map(int, p2))
        cv2.line(img, p1, p2, 255, 2)
        
        if p1 not in vertices:
            vertices.append(p1)
        if p2 not in vertices:
            vertices.append(p2)
            
        edges.add((vertices.index(p1), vertices.index(p2)))
    
    if len(vertices) >= 3:
        contour = np.array(vertices, dtype=np.int32)
        cv2.fillPoly(img, [contour], 255)
    
    return img


#


In [ ]:
geometric_draw = [
        [(200, 200), (300, 200)],
        [(300, 200), (300, 300)],
        [(300, 300), (200, 300)],
        [(200, 300), (250, 225)],
        [(250, 225), (200, 200)],
]
# Ana işlem akışı
img = draw_image(geometric_draw)

plot_images(img, 'Geometric Shape', fig_size=(8, 8))

In [ ]:
fft_image = np.fft.fft2(img)

shifted_fft = np.fft.fftshift(fft_image)



magnitude = np.abs(shifted_fft)

nlog_magnitude = np.log10(magnitude + 1)
angle = np.angle(shifted_fft)


plot_images([nlog_magnitude, angle], ['FFT Magnitude', 'FFT Phase'], fig_size=(15, 15))





In [ ]:
rotate_angles = [0, 30, 210, 300]

rotated_images = []
for angle in rotate_angles:
    rotated_points = rotate_geometric_shape(geometric_draw, angle)
    rotated_img = draw_image(rotated_points)
    rotated_images.append(rotated_img)


titles = [f"rotated by {angle} degrees" for angle in rotate_angles]

plot_images(rotated_images, titles, fig_size=(30, 30))

In [10]:
fft_images = []
for img in rotated_images:
    fft_image = np.fft.fft2(img)
    fft_images.append(fft_image)

shifted_fft_images = [np.fft.fftshift(fft_image) for fft_image in fft_images]

magnitudes = [np.abs(fft_image) for fft_image in shifted_fft_images]

nlog_magnitudes = [np.log10(magnitude + 1) for magnitude in magnitudes]


In [ ]:

titles = [f"rotated by {angle} degrees" for angle in rotate_angles]
plot_images(nlog_magnitudes, titles, fig_size=(30, 30))

In [ ]:
phase_images = [np.angle(fft_image) for fft_image in shifted_fft_images]

plot_images(phase_images, titles, fig_size=(30, 30))

In [ ]:
shifted_points = []
pts = geometric_draw
pts1 = shift_geometric_shape(geometric_draw, -150, -150)
pts2 = shift_geometric_shape(geometric_draw, 150, 150)
shifted_points.append(pts1)
shifted_points.append(pts)
shifted_points.append(pts2)


shifted_images = [draw_image(pts) for pts in shifted_points]

titles = ['shifted by (-150, -150)', 'original', 'shifted by (150, 150)']

plot_images(shifted_images, titles, fig_size=(30, 30))

In [14]:
fft_images = [np.fft.fft2(img) for img in shifted_images]

shifted_fft_images = [np.fft.fftshift(fft_image) for fft_image in fft_images]

magnitudes = [np.abs(fft_image) for fft_image in shifted_fft_images]

nlog_magnitudes = [np.log10(magnitude + 1) for magnitude in magnitudes]

angles = [np.angle(fft_image) for fft_image in fft_images]

titles = ['shifted by (-150, 50)', 'original', 'shifted by (50, -150)']



In [ ]:
plot_images(nlog_magnitudes, titles, fig_size=(30, 30))

In [ ]:
plot_images(angles, titles, fig_size=(30, 30))

In [17]:
def radius_filter(fft_data, radius):
    copy_data = fft_data.copy()
    center = (fft_data.shape[0] // 2, fft_data.shape[1] // 2)
    
    for i in range(fft_data.shape[0]):
        for j in range(fft_data.shape[1]):
            distance = np.sqrt((i - center[0])**2 + (j - center[1])**2)
            if distance > radius:
                copy_data[i, j] = 0
    return copy_data




In [ ]:
radiuses = [10, 50, 100, 300]


image_dir = os.path.join(images_dir, list_images[9])

image = Image.open(image_dir)

image = image.convert('L')

image = np.array(image)

plot_images(image, 'Original Image', fig_size=(8, 8))

In [ ]:
fft_image = np.fft.fft2(image)

shifted_fft = np.fft.fftshift(fft_image)

magnitudes = np.abs(shifted_fft)

nlog_magnitudes = np.log10(magnitudes + 1)

angle = np.angle(fft_image)

plot_images([image, nlog_magnitudes, angle], ['Original Image', 'FFT Magnitude', 'FFT Phase'], fig_size=(30, 30))

In [ ]:
filtered_image = radius_filter(shifted_fft, 10)

filtered_magnitudes = np.abs(filtered_image)

nlog_filtered_magnitudes = np.log10(filtered_magnitudes + 1)

filtered_angle = np.angle(filtered_image)

plot_images([image, nlog_filtered_magnitudes, filtered_angle], ['Original Image', 'Filtered FFT Magnitude', 'Filtered FFT Phase'], fig_size=(30, 30))

In [ ]:
inverse_shifted_fft = np.fft.ifftshift(filtered_image)

inverse_fft = np.fft.ifft2(inverse_shifted_fft)

inversed_image = np.abs(inverse_fft).astype(np.uint8)

plot_images([image, inversed_image], ['Original Image', 'Inversed Image'], fig_size=(15, 15))

In [ ]:
filtered_images = []
for radius in radiuses:
    filtered_image = radius_filter(shifted_fft.copy(), radius)
    filtered_images.append(filtered_image)

filtered_magnitudes = [np.abs(filtered_image) for filtered_image in filtered_images]

nlog_filtered_magnitudes = [np.log10(magnitude + 1) for magnitude in filtered_magnitudes]

filtered_angles = [np.angle(filtered_image) for filtered_image in filtered_images]

titles = [f'Filtered FFT Magnitude (radius={radius})' for radius in radiuses]

plot_images(nlog_filtered_magnitudes, titles, fig_size=(30, 30))

In [ ]:
inverse_shifted_ffts = [np.fft.ifftshift(filtered_image) for filtered_image in filtered_images]

inverse_ffts = [np.fft.ifft2(inverse_shifted_fft) for inverse_shifted_fft in inverse_shifted_ffts]

inversed_images = [np.abs(inverse_fft).astype(np.uint8) for inverse_fft in inverse_ffts]

titles = [f'Inversed Image (radius={radius})' for radius in radiuses]

plot_images(inversed_images, titles, fig_size=(30, 30))